In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

/home/octav/gitrepos/tum-thesis


In [2]:
from otgnn.models import GCN
from otgnn.graph import MolGraph

from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time

In [3]:
sys.path

['/home/octav/anaconda3/envs/mol_ot/lib/python36.zip',
 '/home/octav/anaconda3/envs/mol_ot/lib/python3.6',
 '/home/octav/anaconda3/envs/mol_ot/lib/python3.6/lib-dynload',
 '',
 '/home/octav/anaconda3/envs/mol_ot/lib/python3.6/site-packages',
 '/home/octav/anaconda3/envs/mol_ot/lib/python3.6/site-packages/IPython/extensions',
 '/home/octav/.ipython',
 '/home/octav/gitrepos/tum-thesis/otgnn']

In [4]:
data_loader = get_loader("iclr19-graph2graph/data/qed", 48, True)

In [5]:
for i in data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [13]:
sys.argv = ["", "-cuda"]
args = get_args()
# args.device = "cpu"
gcn = GCN(args).to(args.device)
opt = torch.nn.Linear(50, 50).to(args.device)

In [15]:
args

Namespace(agg_func='sum', batch_norm=False, cuda=True, device='cuda:0', dropout_ffn=0.0, dropout_gcn=0.0, ffn_activation='LeakyReLU', linear_out=False, n_epochs=10, n_ffn_hidden=100, n_hidden=50, n_labels=1, n_layers=5, pc_hidden=50)

In [92]:
gcn = GCN(args).to(args.device)

In [86]:
gcn

GCN(
  (W_message_i): Linear(in_features=100, out_features=50, bias=False)
  (W_message_h): Linear(in_features=50, out_features=50, bias=False)
  (W_message_o): Linear(in_features=143, out_features=50, bias=True)
  (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
  (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
  (dropout_gcn): Dropout(p=0.0, inplace=False)
  (dropout_ffn): Dropout(p=0.0, inplace=False)
)

In [88]:
gcn

GCN(
  (W_message_i): Linear(in_features=100, out_features=50, bias=False)
  (W_message_h): Linear(in_features=50, out_features=50, bias=False)
  (W_message_o): Linear(in_features=143, out_features=50, bias=True)
  (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
  (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
  (dropout_gcn): Dropout(p=0.0, inplace=False)
  (dropout_ffn): Dropout(p=0.0, inplace=False)
)

In [93]:
opt(gcn.forward(Y)[0])

tensor([[-0.1131, -0.0804, -0.0672,  ..., -0.1228,  0.2296, -0.0391],
        [-0.1317,  0.0174, -0.0672,  ..., -0.0214,  0.3100, -0.0278],
        [-0.1112,  0.0032, -0.0945,  ..., -0.0932,  0.1757,  0.0186],
        ...,
        [-0.1082,  0.0235, -0.0526,  ..., -0.0472,  0.3220, -0.0061],
        [-0.1707, -0.0715, -0.1234,  ..., -0.1141,  0.1511, -0.0728],
        [-0.1173,  0.0305, -0.1945,  ..., -0.1928,  0.1837,  0.0267]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [113]:
gcn.forward(X)[0].shape

torch.Size([1041, 50])

In [127]:
print (np.cumsum(([len(x.atoms) for x in X.mols])))

[  22   40   61   79  100  120  144  164  186  205  226  251  276  298
  321  340  360  386  411  433  455  476  500  523  542  566  586  612
  635  659  679  695  719  738  762  781  799  822  844  866  890  914
  938  958  978  998 1021 1041]


In [119]:
x = X.mols[0]

In [159]:
from mol_opt import mol_opt
molopt = mol_opt.MolOpt(args).to(device = args.device)
molopt

MolOpt(
  (GCN): GCN(
    (W_message_i): Linear(in_features=100, out_features=50, bias=False)
    (W_message_h): Linear(in_features=50, out_features=50, bias=False)
    (W_message_o): Linear(in_features=143, out_features=50, bias=True)
    (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
    (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
    (dropout_gcn): Dropout(p=0.0, inplace=False)
    (dropout_ffn): Dropout(p=0.0, inplace=False)
  )
  (opt0): Linear(in_features=50, out_features=50, bias=True)
  (opt1): Linear(in_features=50, out_features=50, bias=True)
  (delta_loss): MSELoss()
)

In [160]:
molopt.align(molopt.encode(X), X, molopt.encode(Y), Y)

tensor([[-0.3370,  0.1239, -0.0026,  ..., -0.0845,  0.1382, -0.0048],
        [-0.3119,  0.0086,  0.1595,  ..., -0.0298,  0.0852,  0.3029],
        [-0.2207, -0.0153,  0.0735,  ..., -0.0120,  0.2595,  0.2476],
        ...,
        [-0.3421,  0.1246,  0.0272,  ..., -0.0622,  0.1342, -0.0053],
        [-0.3372,  0.0805, -0.0215,  ...,  0.0689,  0.2914,  0.3123],
        [-0.2457, -0.0502,  0.0285,  ...,  0.1159,  0.1260,  0.3469]],
       device='cuda:0', grad_fn=<CopySlices>)

In [161]:
molopt.encode(Y)

tensor([[-0.3370,  0.1239, -0.0026,  ..., -0.0845,  0.1382, -0.0048],
        [-0.2441, -0.0096, -0.0594,  ..., -0.0485,  0.1602,  0.1980],
        [-0.0878,  0.1209, -0.0024,  ..., -0.1630, -0.0446,  0.1149],
        ...,
        [-0.2650,  0.0218, -0.0695,  ...,  0.1347,  0.3265,  0.3682],
        [-0.2774,  0.2852, -0.1069,  ...,  0.0081, -0.0275,  0.0836],
        [-0.2457, -0.0502,  0.0285,  ...,  0.1159,  0.1260,  0.3469]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [162]:
molopt.forward_train(X, Y)

tensor(0.0155, device='cuda:0', grad_fn=<MeanBackward0>)

In [163]:
for epoch in range(args.n_epochs):
    start = time.time()
    for idx, i in enumerate(data_loader):
        X = (MolGraph(i[0]))
        Y = (MolGraph(i[1]))

        # create your optimizer
        optimizer = torch.optim.SGD(molopt.parameters(), lr=0.01)

        # in your training loop:
        optimizer.zero_grad()   # zero the gradient buffers
        loss = molopt.forward_train(X, Y)
        loss.backward()
        optimizer.step()    # Does the update

        print ("Iter: {}, loss: {}".format(idx, loss.item()))
    end = time.time()
    print("Time for epoch {}: {}", epoch, end - start)

Iter: 0, loss: 0.015534802339971066
Iter: 1, loss: 0.015233045443892479
Iter: 2, loss: 0.014541471377015114
Iter: 3, loss: 0.014296744018793106
Iter: 4, loss: 0.014862217009067535
Iter: 5, loss: 0.014735095202922821
Iter: 6, loss: 0.014787458814680576
Iter: 7, loss: 0.0148368114605546
Iter: 8, loss: 0.014837524853646755
Iter: 9, loss: 0.014857822097837925
Iter: 10, loss: 0.014195257797837257
Iter: 11, loss: 0.01448412798345089
Iter: 12, loss: 0.01481739804148674
Iter: 13, loss: 0.01585579290986061
Iter: 14, loss: 0.015088125132024288
Iter: 15, loss: 0.014189892448484898
Iter: 16, loss: 0.014590233564376831
Iter: 17, loss: 0.014030560851097107
Iter: 18, loss: 0.014501072466373444
Iter: 19, loss: 0.013706115074455738
Iter: 20, loss: 0.014175619930028915
Iter: 21, loss: 0.014129242859780788
Iter: 22, loss: 0.013829604722559452
Iter: 23, loss: 0.013918042182922363
Iter: 24, loss: 0.01380153838545084
Iter: 25, loss: 0.014579208567738533
Iter: 26, loss: 0.014408345334231853
Iter: 27, loss: 0

/home/octav/anaconda3/envs/mol_ot/lib/python3.6/site-packages/ot/lp/__init__.py:276: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)


Iter: 34, loss: 0.014029206708073616
Iter: 35, loss: 0.013968406245112419
Iter: 36, loss: 0.014084894210100174
Iter: 37, loss: 0.014040748588740826
Iter: 38, loss: 0.014291472733020782
Iter: 39, loss: 0.014438503421843052
Iter: 40, loss: 0.01469825953245163
Iter: 41, loss: 0.013522089459002018
Iter: 42, loss: 0.013808383606374264
Iter: 43, loss: 0.014365884475409985
Iter: 44, loss: 0.013690410181879997
Iter: 45, loss: 0.01380215585231781
Iter: 46, loss: 0.013775057159364223
Iter: 47, loss: 0.013525346294045448
Iter: 48, loss: 0.013605396263301373
Iter: 49, loss: 0.013963649980723858
Iter: 50, loss: 0.013937383890151978
Iter: 51, loss: 0.013569599017500877
Iter: 52, loss: 0.013187862001359463
Iter: 53, loss: 0.013646628707647324
Iter: 54, loss: 0.013621378690004349
Iter: 55, loss: 0.013175789266824722
Iter: 56, loss: 0.013652865774929523
Iter: 57, loss: 0.01333098765462637
Iter: 58, loss: 0.013325710780918598
Iter: 59, loss: 0.013774197548627853
Iter: 60, loss: 0.013378186151385307
Iter

Iter: 5, loss: 0.011062366887927055
Iter: 6, loss: 0.011161036789417267
Iter: 7, loss: 0.011199872009456158
Iter: 8, loss: 0.01121568400412798
Iter: 9, loss: 0.011119906790554523
Iter: 10, loss: 0.010642554610967636
Iter: 11, loss: 0.010950086638331413
Iter: 12, loss: 0.011198685504496098
Iter: 13, loss: 0.012002616189420223
Iter: 14, loss: 0.011378305032849312
Iter: 15, loss: 0.010684579610824585
Iter: 16, loss: 0.010998154990375042
Iter: 17, loss: 0.010535567998886108
Iter: 18, loss: 0.010907185263931751
Iter: 19, loss: 0.010277491994202137
Iter: 20, loss: 0.010675954632461071
Iter: 21, loss: 0.010673644952476025
Iter: 22, loss: 0.010425758548080921
Iter: 23, loss: 0.010501904413104057
Iter: 24, loss: 0.010392596945166588
Iter: 25, loss: 0.011063323356211185
Iter: 26, loss: 0.010888702236115932
Iter: 27, loss: 0.01040828786790371
Iter: 28, loss: 0.011097054928541183
Iter: 29, loss: 0.010628379881381989
Iter: 30, loss: 0.011344282887876034
Iter: 31, loss: 0.010476179420948029
Iter: 32

Iter: 102, loss: 0.0089333551004529
Iter: 103, loss: 0.008979018777608871
Iter: 104, loss: 0.009097672067582607
Iter: 105, loss: 0.008776803500950336
Iter: 106, loss: 0.00881857518106699
Iter: 107, loss: 0.008477690629661083
Iter: 108, loss: 0.008470491506159306
Iter: 109, loss: 0.008933167904615402
Iter: 110, loss: 0.008951665833592415
Iter: 111, loss: 0.00914007518440485
Iter: 112, loss: 0.008857402950525284
Iter: 113, loss: 0.008729237131774426
Iter: 114, loss: 0.008920636959373951
Iter: 115, loss: 0.009286978282034397
Iter: 116, loss: 0.008761647157371044
Iter: 117, loss: 0.00937346089631319
Iter: 118, loss: 0.009057339280843735
Iter: 119, loss: 0.008951058611273766
Iter: 120, loss: 0.008448668755590916
Iter: 121, loss: 0.008493450470268726
Iter: 122, loss: 0.009120439179241657
Iter: 123, loss: 0.008751928806304932
Iter: 124, loss: 0.00897366926074028
29.838321924209595
Iter: 0, loss: 0.009316830895841122
Iter: 1, loss: 0.009152702987194061
Iter: 2, loss: 0.008635593578219414
Iter:

Iter: 73, loss: 0.007536970544606447
Iter: 74, loss: 0.007517963647842407
Iter: 75, loss: 0.007469389121979475
Iter: 76, loss: 0.0075345225632190704
Iter: 77, loss: 0.007199797313660383
Iter: 78, loss: 0.007907253690063953
Iter: 79, loss: 0.007559444755315781
Iter: 80, loss: 0.007300992496311665
Iter: 81, loss: 0.007529605645686388
Iter: 82, loss: 0.00736561045050621
Iter: 83, loss: 0.007269708905369043
Iter: 84, loss: 0.0076103536412119865
Iter: 85, loss: 0.007507013622671366
Iter: 86, loss: 0.007951815612614155
Iter: 87, loss: 0.007180726155638695
Iter: 88, loss: 0.007435615174472332
Iter: 89, loss: 0.007705459836870432
Iter: 90, loss: 0.0073601980693638325
Iter: 91, loss: 0.0073839761316776276
Iter: 92, loss: 0.0074530672281980515
Iter: 93, loss: 0.007299473974853754
Iter: 94, loss: 0.007499695755541325
Iter: 95, loss: 0.007230495568364859
Iter: 96, loss: 0.007363138720393181
Iter: 97, loss: 0.007327426690608263
Iter: 98, loss: 0.00784387905150652
Iter: 99, loss: 0.00759094394743442

Iter: 43, loss: 0.006678000092506409
Iter: 44, loss: 0.006363663822412491
Iter: 45, loss: 0.006386504042893648
Iter: 46, loss: 0.006521744187921286
Iter: 47, loss: 0.006312240846455097
Iter: 48, loss: 0.006283583119511604
Iter: 49, loss: 0.006495081819593906
Iter: 50, loss: 0.00639758724719286
Iter: 51, loss: 0.0064171007834374905
Iter: 52, loss: 0.006138720083981752
Iter: 53, loss: 0.006400314625352621
Iter: 54, loss: 0.006390412803739309
Iter: 55, loss: 0.006179599557071924
Iter: 56, loss: 0.006450003478676081
Iter: 57, loss: 0.006157366558909416
Iter: 58, loss: 0.006233284715563059
Iter: 59, loss: 0.0064674667082726955
Iter: 60, loss: 0.00632502930238843
Iter: 61, loss: 0.00660732202231884
Iter: 62, loss: 0.006467925850301981
Iter: 63, loss: 0.006596527993679047
Iter: 64, loss: 0.0064226905815303326
Iter: 65, loss: 0.0062757087871432304
Iter: 66, loss: 0.006404914893209934
Iter: 67, loss: 0.006311205215752125
Iter: 68, loss: 0.006149180233478546
Iter: 69, loss: 0.006356267724186182


Iter: 13, loss: 0.006290254648774862
Iter: 14, loss: 0.005886174738407135
Iter: 15, loss: 0.005493386182934046
Iter: 16, loss: 0.0056996108032763
Iter: 17, loss: 0.005404071882367134
Iter: 18, loss: 0.005594437941908836
Iter: 19, loss: 0.005218156613409519
Iter: 20, loss: 0.005501819308847189
Iter: 21, loss: 0.005564944818615913
Iter: 22, loss: 0.005394055973738432
Iter: 23, loss: 0.005439731292426586
Iter: 24, loss: 0.00537204509600997
Iter: 25, loss: 0.005793604534119368
Iter: 26, loss: 0.00563765550032258
Iter: 27, loss: 0.0054502300918102264
Iter: 28, loss: 0.005842233542352915
Iter: 29, loss: 0.005558673292398453
Iter: 30, loss: 0.005884532816708088
Iter: 31, loss: 0.005464375484734774
Iter: 32, loss: 0.0055547053925693035
Iter: 33, loss: 0.00588854169473052
Iter: 34, loss: 0.0054968721233308315
Iter: 35, loss: 0.005542881321161985
Iter: 36, loss: 0.0055511388927698135
Iter: 37, loss: 0.005565284751355648
Iter: 38, loss: 0.005689182318747044
Iter: 39, loss: 0.0058484990149736404
I

In [130]:
X.scope

[(0, 22),
 (22, 18),
 (40, 21),
 (61, 18),
 (79, 21),
 (100, 20),
 (120, 24),
 (144, 20),
 (164, 22),
 (186, 19),
 (205, 21),
 (226, 25),
 (251, 25),
 (276, 22),
 (298, 23),
 (321, 19),
 (340, 20),
 (360, 26),
 (386, 25),
 (411, 22),
 (433, 22),
 (455, 21),
 (476, 24),
 (500, 23),
 (523, 19),
 (542, 24),
 (566, 20),
 (586, 26),
 (612, 23),
 (635, 24),
 (659, 20),
 (679, 16),
 (695, 24),
 (719, 19),
 (738, 24),
 (762, 19),
 (781, 18),
 (799, 23),
 (822, 22),
 (844, 22),
 (866, 24),
 (890, 24),
 (914, 24),
 (938, 20),
 (958, 20),
 (978, 20),
 (998, 23),
 (1021, 20)]

In [138]:
x_embedding.narrow(0, 0, 22)

tensor([[-0.0455,  0.0447,  0.0261,  ..., -0.0160,  0.0526,  0.0204],
        [-0.0465,  0.0597,  0.0177,  ..., -0.1785, -0.0618,  0.0922],
        [ 0.0161,  0.0673, -0.1091,  ...,  0.0081, -0.0677,  0.1738],
        ...,
        [ 0.0850, -0.0669, -0.0290,  ..., -0.0392, -0.0300,  0.0311],
        [ 0.1073, -0.0826, -0.0462,  ..., -0.0227, -0.0654,  0.0300],
        [ 0.0796,  0.0683,  0.0362,  ..., -0.0448, -0.0863,  0.0600]],
       device='cuda:0', grad_fn=<SliceBackward>)

In [144]:
x_embedding[0:22,:]

torch.Size([22, 50])

In [136]:
x_embedding.narrow(0, 22, 18)

tensor([[-9.2475e-02,  4.4592e-02,  4.1157e-02, -6.2958e-02, -2.6449e-02,
          5.1672e-03, -4.6384e-02, -5.3797e-02, -7.3464e-02, -7.3005e-02,
         -1.7704e-02,  8.3490e-02, -1.3573e-01,  8.2092e-03, -5.4852e-02,
         -1.0005e-01,  1.0265e-01,  8.8467e-02,  1.8191e-02,  8.6351e-02,
          1.7423e-01,  2.4751e-02,  6.3913e-03,  3.5624e-03,  8.4073e-03,
         -2.3538e-02,  1.3579e-01,  4.8769e-02,  1.5200e-01, -2.5218e-02,
          2.4999e-01, -7.4404e-03,  6.2257e-02, -1.2847e-01, -1.4130e-01,
          1.4734e-01, -2.1842e-03,  8.3792e-02, -8.7801e-02,  4.2352e-02,
          2.5093e-01, -3.2312e-02,  7.7363e-02,  6.2747e-02,  1.1126e-01,
          6.4715e-02, -3.6868e-02,  1.1756e-03,  7.7242e-03,  7.6595e-03],
        [ 2.0119e-03, -1.1420e-01, -6.6094e-03, -6.3875e-02,  8.7873e-02,
          1.7288e-02,  9.8871e-02, -1.4052e-01, -5.1341e-02, -1.6861e-01,
         -7.1023e-02,  3.3994e-02,  2.1864e-02, -9.5770e-03,  4.5321e-02,
          1.1468e-01,  1.7144e-01,  1

In [21]:
start = time.time()
for idx, i in enumerate(data_loader):
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    
    # create your optimizer
    optimizer = torch.optim.SGD(molopt.parameters(), lr=0.01)

    # in your training loop:
    optimizer.zero_grad()   # zero the gradient buffers
    loss = molopt.forward_train(X, Y)
    loss.backward()
    optimizer.step()    # Does the update

    print (idx, loss)
end = time.time()
print(end - start)

0 tensor(0.0229, device='cuda:0', grad_fn=<MeanBackward0>)
1 tensor(0.0221, device='cuda:0', grad_fn=<MeanBackward0>)
2 tensor(0.0220, device='cuda:0', grad_fn=<MeanBackward0>)
3 tensor(0.0217, device='cuda:0', grad_fn=<MeanBackward0>)
4 tensor(0.0227, device='cuda:0', grad_fn=<MeanBackward0>)
5 tensor(0.0220, device='cuda:0', grad_fn=<MeanBackward0>)
6 tensor(0.0214, device='cuda:0', grad_fn=<MeanBackward0>)
7 tensor(0.0216, device='cuda:0', grad_fn=<MeanBackward0>)
8 tensor(0.0213, device='cuda:0', grad_fn=<MeanBackward0>)
9 tensor(0.0206, device='cuda:0', grad_fn=<MeanBackward0>)
10 tensor(0.0207, device='cuda:0', grad_fn=<MeanBackward0>)
11 tensor(0.0208, device='cuda:0', grad_fn=<MeanBackward0>)
12 tensor(0.0221, device='cuda:0', grad_fn=<MeanBackward0>)
13 tensor(0.0214, device='cuda:0', grad_fn=<MeanBackward0>)
14 tensor(0.0210, device='cuda:0', grad_fn=<MeanBackward0>)
15 tensor(0.0206, device='cuda:0', grad_fn=<MeanBackward0>)
16 tensor(0.0205, device='cuda:0', grad_fn=<MeanBa

# run the implemented function

In [108]:
args = get_args()
args.n_epochs = 10
args

Namespace(agg_func='sum', batch_norm=False, cuda=True, device='cuda:0', dropout_ffn=0.0, dropout_gcn=0.0, ffn_activation='LeakyReLU', linear_out=False, n_epochs=10, n_ffn_hidden=100, n_hidden=50, n_labels=1, n_layers=5, pc_hidden=50)

In [110]:
molopt = main(args, molopt, data_loader)

Epoch: 0
Iter: 0, loss: 0.018722232431173325
Iter: 1, loss: 0.017757058143615723
Iter: 2, loss: 0.018040090799331665
Iter: 3, loss: 0.01792227476835251
Iter: 4, loss: 0.018236974254250526
Iter: 5, loss: 0.018043730407953262
Iter: 6, loss: 0.018231509253382683
Iter: 7, loss: 0.01780024543404579
Iter: 8, loss: 0.017768017947673798
Iter: 9, loss: 0.016536850482225418
Iter: 10, loss: 0.01756877638399601
Iter: 11, loss: 0.017272545024752617
Iter: 12, loss: 0.018287181854248047
Iter: 13, loss: 0.017411379143595695
Iter: 14, loss: 0.01773667335510254
Iter: 15, loss: 0.01723826862871647
Iter: 16, loss: 0.016550524160265923
Iter: 17, loss: 0.017243877053260803
Iter: 18, loss: 0.016688283532857895
Iter: 19, loss: 0.017896555364131927
Iter: 20, loss: 0.016977950930595398
Iter: 21, loss: 0.016956424340605736
Iter: 22, loss: 0.017922870814800262
Iter: 23, loss: 0.017049843445420265
Iter: 24, loss: 0.017049923539161682
Iter: 25, loss: 0.017336567863821983
Iter: 26, loss: 0.017349278554320335
Iter: 2

Iter: 97, loss: 0.011339572258293629
Iter: 98, loss: 0.011447019875049591
Iter: 99, loss: 0.011545014567673206
Iter: 100, loss: 0.011522823013365269
Iter: 101, loss: 0.011801736429333687
Iter: 102, loss: 0.011706486344337463
Iter: 103, loss: 0.011780024506151676
Iter: 104, loss: 0.011419154703617096
Iter: 105, loss: 0.01167968474328518
Iter: 106, loss: 0.011806605383753777
Iter: 107, loss: 0.011295836418867111
Iter: 108, loss: 0.011602933518588543
Iter: 109, loss: 0.011758827604353428
Iter: 110, loss: 0.011267127469182014
Iter: 111, loss: 0.011211936362087727
Iter: 112, loss: 0.011436959728598595
Iter: 113, loss: 0.01112006139010191
Iter: 114, loss: 0.011402169242501259
Iter: 115, loss: 0.011062969453632832
Iter: 116, loss: 0.010979493148624897
Iter: 117, loss: 0.011338317766785622
Iter: 118, loss: 0.011337358504533768
Iter: 119, loss: 0.01176669541746378
Iter: 120, loss: 0.01149203535169363
Iter: 121, loss: 0.011538762599229813
Iter: 122, loss: 0.011304404586553574
Iter: 123, loss: 0.

Iter: 67, loss: 0.00887337327003479
Iter: 68, loss: 0.008852193132042885
Iter: 69, loss: 0.008837568573653698
Iter: 70, loss: 0.008760564960539341
Iter: 71, loss: 0.008617549203336239
Iter: 72, loss: 0.008941447362303734
Iter: 73, loss: 0.009031088091433048
Iter: 74, loss: 0.008357178419828415
Iter: 75, loss: 0.00862843543291092
Iter: 76, loss: 0.00871327891945839
Iter: 77, loss: 0.008617350831627846
Iter: 78, loss: 0.008771123364567757
Iter: 79, loss: 0.008541226387023926
Iter: 80, loss: 0.008450784720480442
Iter: 81, loss: 0.008678791113197803
Iter: 82, loss: 0.008580921217799187
Iter: 83, loss: 0.008684307336807251
Iter: 84, loss: 0.00868390966206789
Iter: 85, loss: 0.008667011745274067
Iter: 86, loss: 0.008426375687122345
Iter: 87, loss: 0.008933648467063904
Iter: 88, loss: 0.008654678240418434
Iter: 89, loss: 0.008703500032424927
Iter: 90, loss: 0.00869210995733738
Iter: 91, loss: 0.008731120266020298
Iter: 92, loss: 0.008545096963644028
Iter: 93, loss: 0.008616551756858826
Iter: 

Iter: 36, loss: 0.006943654268980026
Iter: 37, loss: 0.007237760350108147
Iter: 38, loss: 0.007261067628860474
Iter: 39, loss: 0.006926509086042643
Iter: 40, loss: 0.007037026807665825
Iter: 41, loss: 0.007225599605590105
Iter: 42, loss: 0.006966614630073309
Iter: 43, loss: 0.007298057898879051
Iter: 44, loss: 0.007183251436799765
Iter: 45, loss: 0.007109316997230053
Iter: 46, loss: 0.0073044574819505215
Iter: 47, loss: 0.007288237102329731
Iter: 48, loss: 0.006847213488072157
Iter: 49, loss: 0.007197159808129072
Iter: 50, loss: 0.00712421303614974
Iter: 51, loss: 0.007385751698166132
Iter: 52, loss: 0.00685857841745019
Iter: 53, loss: 0.0070550041273236275
Iter: 54, loss: 0.007219631224870682
Iter: 55, loss: 0.00681339344009757
Iter: 56, loss: 0.0071955774910748005
Iter: 57, loss: 0.006944354623556137
Iter: 58, loss: 0.0071913860738277435
Iter: 59, loss: 0.007017304189503193
Iter: 60, loss: 0.006772950757294893
Iter: 61, loss: 0.007211155258119106
Iter: 62, loss: 0.007061890326440334


Iter: 4, loss: 0.005956504959613085
Iter: 5, loss: 0.006206505466252565
Iter: 6, loss: 0.006158994510769844
Iter: 7, loss: 0.006321229971945286
Iter: 8, loss: 0.006258407142013311
Iter: 9, loss: 0.005867192056030035
Iter: 10, loss: 0.005944552831351757
Iter: 11, loss: 0.005935914348810911
Iter: 12, loss: 0.006349125411361456
Iter: 13, loss: 0.006305607967078686
Iter: 14, loss: 0.006304513197392225
Iter: 15, loss: 0.005967371165752411
Iter: 16, loss: 0.005892177578061819
Iter: 17, loss: 0.005989509169012308
Iter: 18, loss: 0.006001384928822517
Iter: 19, loss: 0.006092861294746399
Iter: 20, loss: 0.00605185329914093
Iter: 21, loss: 0.005936257541179657
Iter: 22, loss: 0.006147680804133415
Iter: 23, loss: 0.0060217492282390594
Iter: 24, loss: 0.006027007941156626
Iter: 25, loss: 0.006092998664826155
Iter: 26, loss: 0.0062633659690618515
Iter: 27, loss: 0.00607368303462863
Iter: 28, loss: 0.006102191749960184
Iter: 29, loss: 0.005827110260725021
Iter: 30, loss: 0.006162941921502352
Iter: 3

Iter: 99, loss: 0.005205238237977028
Iter: 100, loss: 0.005162783898413181
Iter: 101, loss: 0.005496329162269831
Iter: 102, loss: 0.005427616182714701
Iter: 103, loss: 0.0053450074046850204
Iter: 104, loss: 0.005186091177165508


KeyboardInterrupt: 

# try out some results

In [164]:
for i in data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [178]:
x_embedding, x_delta = molopt.forward(X)
y_embedding = molopt.encode(Y)
y_aligned = molopt.align(x_embedding, X, y_embedding, Y)
xhat_delta = molopt.delta(x_embedding, y_aligned)

In [172]:
x_embedding.shape

torch.Size([1041, 50])

In [173]:
x_delta

tensor([[-0.0329,  0.0609, -0.0998,  ...,  0.0075,  0.0955,  0.0417],
        [-0.0989,  0.0230, -0.0382,  ...,  0.0112,  0.0739,  0.0399],
        [-0.0075,  0.0569, -0.0903,  ..., -0.0105,  0.1205,  0.0362],
        ...,
        [-0.0320,  0.0622, -0.1089,  ...,  0.0226,  0.0922,  0.0389],
        [-0.0710,  0.0140, -0.0761,  ..., -0.0365,  0.0978,  0.0615],
        [-0.0357,  0.0151, -0.0686,  ..., -0.0102,  0.0744,  0.0069]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [179]:
xhat_delta

tensor([[-5.3743e-02,  4.4010e-02, -3.0419e-02,  ..., -3.1573e-02,
          3.3295e-02,  5.1158e-02],
        [-1.7625e-01,  1.2730e-01, -6.4805e-02,  ...,  5.5192e-02,
          1.4270e-01, -6.1634e-03],
        [-7.5377e-02, -1.6244e-02,  5.7163e-02,  ...,  5.2657e-02,
          6.2681e-02,  2.1037e-02],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.9689e-01, -5.8483e-02,  9.4051e-03,  ...,  1.4330e-01,
          3.4291e-01,  1.2687e-01],
        [-8.1267e-03,  1.4335e-04, -5.4330e-04,  ..., -4.7268e-03,
          1.3943e-02, -8.6641e-03]], device='cuda:0', grad_fn=<SubBackward0>)

In [180]:
x_delta - xhat_delta

tensor([[ 0.0209,  0.0169, -0.0694,  ...,  0.0390,  0.0622, -0.0094],
        [ 0.0773, -0.1043,  0.0266,  ..., -0.0440, -0.0688,  0.0460],
        [ 0.0679,  0.0732, -0.1474,  ..., -0.0631,  0.0578,  0.0152],
        ...,
        [-0.0320,  0.0622, -0.1089,  ...,  0.0226,  0.0922,  0.0389],
        [ 0.1259,  0.0725, -0.0856,  ..., -0.1798, -0.2451, -0.0654],
        [-0.0276,  0.0149, -0.0681,  ..., -0.0054,  0.0605,  0.0156]],
       device='cuda:0', grad_fn=<SubBackward0>)

In [78]:
molopt.delta(x_embedding, y) - xhat

tensor([[ 0.0016, -0.0741, -0.0745,  ...,  0.0228, -0.0114, -0.0270],
        [ 0.0706, -0.2145, -0.2110,  ...,  0.3129,  0.1174, -0.0711],
        [-0.1300, -0.2984,  0.1859,  ..., -0.0819,  0.1519, -0.2849],
        ...,
        [ 0.2385, -0.1195, -0.1575,  ...,  0.0702, -0.1347,  0.0785],
        [ 0.0352,  0.1964, -0.0075,  ..., -0.0421,  0.1371, -0.1475],
        [-0.0727,  0.1660,  0.0481,  ...,  0.0867,  0.1587, -0.0863]],
       device='cuda:0', grad_fn=<SubBackward0>)

In [35]:
Y.mols[]